# Library

In [2]:
import os 
import glob
import re

# Propocess

In [48]:
def checkfile(outfile) :
    if os.path.isfile(outfile) == True :
        print('already file %s!'%outfile)
        os.remove(outfile)

def readfile(filename) :
    with open(filename, 'r', encoding='utf8') as read :
        data = read.readlines()
    print('open file %s'%filename)
    return data

def writefile(arr,outfile) :
    with open(outfile, "a", encoding="utf-8") as wf :
        wf.write(arr + '\n')

def condition() :
    con = {    
            "<left_parenthesis>" : "(",
            "<right_parenthesis>" : ")",
            "<minus>" : "-",
            "<less_than>" : "<",
            "<greater_than>" : ">",
            "<question_mark>" : "?",
            "<quote>" : '"',
            "<plus>" : "+",
            "<exclamation>" : "!",
            "<full_stop>" : ".",
            "<ampersand>" : "&",
            "quote" : '"',
            "<quotation>" : '"'
            }
    return con

def preprocess(infile=None, outfile=None, types=None) :
    if types == None or infile == None :
        print('None file or None type')
        return None
    data = readfile(infile)
    arr = [[]]
    ex = []
    count = 0
    for i in range(1,len(data)) :   
        e = data[i][:-1]
        e = re.sub(' +','\t',e)
        for word,initail in condition().items() :
            e = e.replace(word, initail) 
        if types == 2 :
            e = e.split("\t")[0]
        if e == 'EOS' :
            count += 1
            if i != len(data)-1 :
                arr.insert(count,[])
            ex.append(i)
        elif ex[-1:] == [i-1] :pass
        else :arr[count].append(e)
    checkfile(outfile)
    if arr[-1] != [] :arr[-1].pop()
    for i in arr :
        writefile('%s' % '|'.join(map(str, i)), outfile) 
    print('write file complete!')

def preprocess_multi_files(fdname=None, fdoutput=None, types=None) :
    if types == None or fdname == None :
        print('None file or None type')
        return None
    if fdoutput == None : fdoutput = 'output'
    readfile = glob.glob(fdname+'/*.txt')
    readfd = glob.glob(fdname+'/'+fdoutput)
    if readfd == []:
        print('None file in folder')
        mkfd = os.mkdir(fdname+'/'+fdoutput)
    for rf in readfile :
        print(rf)
        preprocess(rf, fdname+'/'+fdoutput+'/'+rf[11:-4]+'.txt', types)


# (foldername with file, foldername for keep outputfile, types 1(with NER) 2(without NER))
preprocess_multi_files(fdname='preprocess', fdoutput='output', types=1)
# or
# (inputfilename, outputfilename, types)
preprocess(infile='dataset_pre.txt', outfile='outfile_pre_1.txt', types=1)
preprocess(infile='dataset_pre.txt', outfile='outfile_pre_2.txt', types=2)

open file dataset_pre.txt
already file outfile_pre_1.txt!
write file complete!
open file dataset_pre.txt
already file outfile_pre_2.txt!
write file complete!


# Postprocess

In [46]:
def checkfile(outfile) :
    if os.path.isfile(outfile) == True :
        print('already file %s!'%outfile)
        os.remove(outfile)

def readfile(filename) :
    with open(filename, 'r', encoding='utf8') as read :
        data = read.readlines()
    print('open file %s'%filename)
    return data

def writefile(arr,outfile) :
    with open(outfile, "a", encoding="utf-8") as wf :
        wf.write(arr + '\n')

def condition() :
    con = {    
            "<left_parenthesis>" : "(\t",
            "<right_parenthesis>" : ")\t",
            "<minus>" : "-\t",
            "<less_than>" : "<\t",
            "<greater_than>" : "\t>",
            "<question_mark>" : "?\t",
            "<quote>" : '"\t',
            "<plus>" : "+\t",
            "<exclamation>" : "!\t",
            "<ampersand>" : "&\t",
            "quote" : '"\t'
            }
    return con

def postprocess(infile, outfile, orifile) :
    orifile = readfile(orifile)    
    file = readfile(infile)
    arr = []
    arr1 = []
    ex = [0]
    count = 0
    for i in range(len(file)) :
        text = file[i][:-1]
        text = text.split('|')
        for spl in text :
            arr.append(spl)
    for len_text in range(len(orifile)) :
        if orifile[len_text] == 'EOS' :
            ex.append(len_text+1)
            arr1.append(arr[count:len_text+1])
            break
        text1 = orifile[len_text][:-1]
        for word,initail in condition().items() :
            text1 = text1.replace(initail, word)
        if [len_text] == ex[-1:] :
            arr1.append(text1)
            pass
        elif text1 == 'EOS' :
            ex.append(len_text+1)
            arr1.append('EOS')
        else :
            arr1.append(text1+'/'+arr[count].split('/')[1])
            count += 1
    checkfile(outfile)
    for na in arr1:
        writefile('%s' % ''.join(map(str, na)), outfile) 


# (inputfilename, outputfilename, originalfilename before preprocess)
# example
postprocess(infile='dataset_post_infile_1.txt', outfile='outfile_post_1.txt', orifile='dataset_post_ori.txt')
postprocess(infile='dataset_post_infile_2.txt', outfile='outfile_post_2.txt', orifile='dataset_post_ori.txt')

open file dataset_post_ori.txt
open file dataset_post_infile_1.txt
already file outfile_post_1.txt!
open file dataset_post_ori.txt
open file dataset_post_infile_2.txt
already file outfile_post_2.txt!
